# Week 7 – 信用贷款决策公平性分析项目

本 Notebook 通过 COMPAS 数据集展示模型训练与公平性分析，同时总结了部分挑战与反思。

## 1. 数据加载与预处理

**目标**：读取 ProPublica 数据，选取目标变量和特征，并拆分敏感属性。

### 挑战与反思
- **敏感属性数据可得性**：实际数据中敏感信息可能缺失或不明确；
- **数据不平衡**：目标变量可能存在类别不平衡问题，需注意分层采样。

In [ ]:
# 读取数据
df = pd.read_csv("/Users/xiangxiaoxin/Documents/GitHub/profile_intro_datascience/week6_classification_decisiontree/data/compas-scores-raw.csv")
print("数据列名:", df.columns)
display(df.head())

# 计算 Age
df["Age"] = pd.to_datetime(df["DateOfBirth"]).dt.year
current_year = pd.Timestamp.now().year
df["Age"] = current_year - df["Age"]

# 构造特征 X 和目标变量 y
X = df[["Age", "RawScore", "Sex_Code_Text", "Ethnic_Code_Text"]]
y, _ = pd.factorize(df["ScoreText"])

# 对类别特征做独热编码
X = pd.get_dummies(X, columns=["Sex_Code_Text", "Ethnic_Code_Text"], drop_first=True)

###############################################
# 缩放敏感特征
###############################################
sensitive_cols = [col for col in X.columns if col.startswith("Sex_Code_Text_") or col.startswith("Ethnic_Code_Text_")]
for col in sensitive_cols:
    X[col] = X[col] * 0.5

X.head()

### 说明
这里我们先对 `DateOfBirth` 计算出 `Age`，并对性别和族裔做了独热编码。随后，我们尝试对敏感属性（性别、族裔）乘以 0.5，理论上想弱化它们在某些算法中对距离计算的影响（但决策树不敏感）。

**反思**：缩放在决策树中可能不起明显作用，但在 KNN 等模型中可能有助于平衡各特征的影响。

## 2. 标准化与数据集划分

**目标**：对所有特征进行标准化，再划分训练集和测试集，确保数据分布均匀。

### 挑战与反思
- 标准化处理确保各特征在同一尺度，但如果先缩放后标准化，缩放效果可能会被抵消；
- 分层采样可缓解数据不平衡问题。

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.3, random_state=42
)
print("训练集形状:", X_train.shape, " 测试集形状:", X_test.shape)

## 3. 模型训练 (KNN 与 Decision Tree)

**目标**：训练基于 KNN 和决策树的模型，查看各自的准确率和混淆矩阵。

### 挑战与反思
- **KNN**：对数值范围敏感，缩放可能有帮助；
- **决策树**：主要基于信息增益选择分裂点，数值缩放对分裂顺序影响较小。
- **模型性能**：关注整体准确率同时也要关注对少数类的识别情况。

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
knn_pred = knn.predict(X_test)

print("KNN Accuracy:", accuracy_score(y_test, knn_pred))
print("KNN Confusion Matrix:\n", confusion_matrix(y_test, knn_pred))
print("KNN Classification Report:\n", classification_report(y_test, knn_pred))

### 说明
KNN 模型在这个数据集上表现不错，但需要特别注意是否只预测多数类（例如 -1 类样本较少导致识别不足）。

## 4. 决策树模型

**目标**：训练决策树，查看分裂结构和模型性能。

### 挑战与反思
- 限制树深 (`max_depth=5`) 是为了防止过拟合；
- 观察决策树的分裂顺序，了解模型最依赖哪些特征（例如 RawScore），以及敏感特征是否仅出现在中下层；
- 关注少数类（例如 -1 类）的预测情况。

In [ ]:
from sklearn.tree import DecisionTreeClassifier, plot_tree

dt = DecisionTreeClassifier(max_depth=5, random_state=42)
dt.fit(X_train, y_train)
dt_pred = dt.predict(X_test)

# 打印决策树结构
def print_tree_details(clf, feature_names, node_index=0, depth=0):
    """
    递归打印决策树每个节点的详细信息:
    - 特征名称 (feature)
    - 阈值 (threshold)
    - Gini impurity
    - 样本数 (n_node_samples)
    - value (各类别样本数)
    """
    left_child = clf.tree_.children_left[node_index]
    right_child = clf.tree_.children_right[node_index]
    threshold = clf.tree_.threshold[node_index]
    feature = clf.tree_.feature[node_index]
    impurity = clf.tree_.impurity[node_index]
    n_samples = clf.tree_.n_node_samples[node_index]
    value = clf.tree_.value[node_index]

    indent = "  " * depth

    if left_child == -1 and right_child == -1:
        print(f"{indent}Leaf node {node_index}:")
        print(f"{indent}  gini = {impurity:.3f}, samples = {n_samples}, value = {value}")
    else:
        print(f"{indent}Node {node_index}:")
        print(f"{indent}  If {feature_names[feature]} <= {threshold:.3f} "
              f"(gini = {impurity:.3f}, samples = {n_samples}, value = {value}):")
        print_tree_details(clf, feature_names, left_child, depth + 1)
        print_tree_details(clf, feature_names, right_child, depth + 1)

print("\n===== Decision Tree Structure (RawScore + Scaled Sensitive) =====")
feature_names = X.columns.tolist()
print_tree_details(dt, feature_names)

print("\nDecision Tree Accuracy:", accuracy_score(y_test, dt_pred))
print("Decision Tree Confusion Matrix:\n", confusion_matrix(y_test, dt_pred))
print("Decision Tree Classification Report:\n", classification_report(y_test, dt_pred))

### 说明
通过 `print_tree_details()` 我们可以直观看到决策树的分裂过程：
- 如果根节点和高层节点主要以 `RawScore` 作为分裂依据，说明该特征信息量最高；
- 如果敏感特征（如性别、族裔）只在后续层次出现，表示它们在模型中辅助细分，但不是主要依据。

**反思**：观察树的结构有助于理解模型决策过程以及识别潜在偏见风险。

## 5. 决策树可视化

使用 `plot_tree()` 绘制决策树，直接在 Notebook 中显示图表。

In [ ]:
plt.figure(figsize=(15, 8), dpi=200)
plot_tree(
    dt,
    feature_names=feature_names,
    class_names=[f"Class_{c}" for c in np.unique(y)],
    filled=True
)
plt.show()

## 6. 模型公平性评估与可视化

利用 Fairlearn 分组指标、ROC 曲线和性能-公平性散点图，对不同模型（基线、后处理、约束）进行对比。

### 挑战与反思
- **公平性与性能的权衡**：在减少不同群体之间的误差差异时，可能牺牲部分整体准确率；
- **指标选择**：需关注 FPR、FNR、Equalized Odds 等指标来判断模型偏差；
- **多模型对比**：不同方法下的公平性与性能表现不同，需要综合考虑。

In [ ]:
# (A) 分布图
fig_dist = px.histogram(
    data, x="Two_yr_Recidivism", color=sensitive_name, barmode="overlay",
    histnorm='density', opacity=0.6,
    title="两年内再犯分布（按非裔 vs 非非裔）"
)
fig_dist.update_layout(xaxis_title="Two_yr_Recidivism", yaxis_title="Density")
fig_dist.show()

# (B) 柱状图：基线模型分组指标
group_metrics = metric_frame_baseline.by_group.reset_index()
fig_bar = go.Figure()
for metric in ["accuracy", "fpr", "fnr"]:
    fig_bar.add_trace(go.Bar(
        x=group_metrics[sensitive_name].astype(str),
        y=group_metrics[metric],
        name=metric
    ))
fig_bar.update_layout(
    barmode="group",
    title="基线模型：不同群体的性能指标",
    xaxis_title=f"{sensitive_name} (0=非, 1=是)",
    yaxis_title="指标值"
)
fig_bar.show()

# (C) ROC 曲线：基线 vs 后处理
fpr_baseline, tpr_baseline, _ = roc_curve(y_test, y_pred_proba_baseline)
if np.any(y_pred_proba_post):
    fpr_post, tpr_post, _ = roc_curve(y_test, y_pred_proba_post)
else:
    fpr_post, tpr_post = [], []

fig_roc = go.Figure()
fig_roc.add_trace(go.Scatter(
    x=fpr_baseline, y=tpr_baseline, mode='lines',
    name=f"Baseline (AUC={auc_baseline:.2f})"
))
if len(fpr_post) > 0:
    fig_roc.add_trace(go.Scatter(
        x=fpr_post, y=tpr_post, mode='lines',
        name=f"Postprocessing (AUC={auc_post:.2f})"
    ))
fig_roc.update_layout(
    title="ROC 曲线对比",
    xaxis_title="False Positive Rate",
    yaxis_title="True Positive Rate"
)
fig_roc.show()

# (D) 性能-公平性散点图
fig_scatter = px.scatter(
    results_df, x="Error_Rate", y="Equalized_Odds", text="Model",
    title="模型性能与公平性权衡图",
    labels={"Error_Rate": "错误率 (1 - Accuracy)", "Equalized_Odds": "Equalized Odds 差异"}
)
fig_scatter.update_traces(textposition='top center')
fig_scatter.show()

## 7. 总结与反思

### 实验总结

1. **RawScore 的作用**：
   - 在本数据中，RawScore 是区分度最高的特征，几乎所有高层分裂都依赖它；
   - 移除 RawScore 后模型准确率大幅下降，说明其它特征不足以弥补其信息量。

2. **敏感特征的缩放**：
   - 虽然理论上缩放敏感特征可能会减弱它们在某些算法中的影响，但决策树主要基于信息增益做分裂，缩放操作对决策树影响不大；
   - 这说明决策树对数值范围不敏感，关键在于哪个特征能降低 Gini impurity 最多。

3. **公平性与性能的权衡**：
   - 后处理和训练时约束方法能在一定程度上降低不同群体的误差差异，但整体准确率可能会有所下降；
   - 实际应用中需要根据业务需求和法律规定找到合适的平衡点。

### 挑战与反思

#### 数据与特征层面
- 敏感属性的获取与标注是公平性分析的重要前提；
- 数据不平衡问题需要采样策略来平衡正负样本；
- 特征工程时需要谨慎处理敏感属性，防止不必要的偏见传递。

#### 模型训练与公平性方法
- 后处理（ThresholdOptimizer）与训练时约束（ExponentiatedGradient）各有优劣，需多次对比；
- 公平性指标的选择（如 Equalized Odds）要与实际应用场景匹配；
- 在追求公平的同时，可能会牺牲一定性能，如何平衡需要实践验证。

#### 可视化与工具
- 可视化工具（Plotly、Matplotlib）有助于直观展示模型决策逻辑和不同群体指标差异；
- 清晰的图表能帮助与非技术人员沟通模型性能与公平性之间的权衡。

#### 部署与合规
- 上线时如何处理敏感属性与公平性问题，需要与法律合规部门充分沟通；
- 模型卡的编写能提升透明度，便于外部审计和持续改进。

### 总体反思

本项目不仅展示了如何使用 Fairlearn 评估和改进模型公平性，也让我认识到：
- 数据科学不仅关注预测准确率，还要考虑公平性、可解释性和社会伦理；
- 每个环节（数据预处理、模型训练、指标评估、可视化）的细节都可能影响最终结果；
- 在实际应用中，应不断与业务、法律和伦理部门协作，找到合适的平衡点。

祝大家在数据科学和公平性探索上不断进步！